In [1]:
import numpy as np
import pickle
import cv2, os
from mnist import MNIST
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
K.set_image_dim_ordering('tf')

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_of_classes = 26+1
image_x, image_y = 28, 28

In [9]:
def step_decay(epoch):
    if epoch < 25:
        lr = 0.01
    elif epoch < 50:
        lr = 0.001
    elif epoch < 75:
        lr = 0.0001
    else: 
        lr = 0.00001
    print(epoch, lr)
    return lr

In [10]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (5,5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (10,10), activation='relu'))
    model.add(MaxPooling2D(pool_size=(7, 7), strides=(7, 7), padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))
    sgd = optimizers.SGD(lr=1e-2)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    filepath="cnn_model_keras2.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    lrate = LearningRateScheduler(step_decay)
    #checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint1, lrate]
    return model, callbacks_list

In [5]:
emnist_data = MNIST(path = 'emnist_source_files\\', return_type = 'numpy')
emnist_data.select_emnist('letters')
train_images, train_labels = emnist_data.load_training()
test_images, test_labels = emnist_data.load_testing()

In [6]:
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_images.shape)

(124800, 784) (124800,)
(20800, 784) (20800, 784)


In [11]:
def train():
    train_images1 = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
    test_images1 = np.reshape(test_images, (test_images.shape[0], image_x, image_y, 1))
    train_labels1 = np_utils.to_categorical(train_labels)
    test_labels1 = np_utils.to_categorical(test_labels)

    model, callbacks_list = cnn_model()
    model.fit(train_images1, train_labels1, validation_data=(test_images1, test_labels1), epochs=100, batch_size=100, callbacks=callbacks_list)
    scores = model.evaluate(test_images1, test_labels1, verbose=0)
    print("CNN Error: %.2f%%" % (100-scores[1]*100))

In [12]:
train()

Train on 124800 samples, validate on 20800 samples
Epoch 1/100
0 0.01
124800/124800 [==============================] - 11s 85us/step - loss: 0.9451 - acc: 0.7556 - val_loss: 0.3071 - val_acc: 0.8999

Epoch 00001: val_acc improved from -inf to 0.89986, saving model to cnn_model_keras2.h5
Epoch 2/100
1 0.01
124800/124800 [==============================] - 10s 83us/step - loss: 0.3334 - acc: 0.8937 - val_loss: 0.2822 - val_acc: 0.9085

Epoch 00002: val_acc improved from 0.89986 to 0.90846, saving model to cnn_model_keras2.h5
Epoch 3/100
2 0.01
124800/124800 [==============================] - 10s 84us/step - loss: 0.2714 - acc: 0.9116 - val_loss: 0.2379 - val_acc: 0.9225

Epoch 00003: val_acc improved from 0.90846 to 0.92250, saving model to cnn_model_keras2.h5
Epoch 4/100
3 0.01
124800/124800 [==============================] - 10s 84us/step - loss: 0.2381 - acc: 0.9218 - val_loss: 0.2301 - val_acc: 0.9253

Epoch 00004: val_acc improved from 0.92250 to 0.92529, saving model to cnn_model_ke

124800/124800 [==============================] - 10s 81us/step - loss: 0.0508 - acc: 0.9801 - val_loss: 0.2715 - val_acc: 0.9363

Epoch 00040: val_acc did not improve
Epoch 41/100
40 0.001
124800/124800 [==============================] - 10s 81us/step - loss: 0.0506 - acc: 0.9804 - val_loss: 0.2706 - val_acc: 0.9366

Epoch 00041: val_acc did not improve
Epoch 42/100
41 0.001
124800/124800 [==============================] - 10s 81us/step - loss: 0.0499 - acc: 0.9808 - val_loss: 0.2729 - val_acc: 0.9366

Epoch 00042: val_acc did not improve
Epoch 43/100
42 0.001
124800/124800 [==============================] - 10s 82us/step - loss: 0.0494 - acc: 0.9806 - val_loss: 0.2723 - val_acc: 0.9360

Epoch 00043: val_acc did not improve
Epoch 44/100
43 0.001
124800/124800 [==============================] - 10s 83us/step - loss: 0.0492 - acc: 0.9809 - val_loss: 0.2739 - val_acc: 0.9362

Epoch 00044: val_acc did not improve
Epoch 45/100
44 0.001
124800/124800 [==============================] - 10s 82

124800/124800 [==============================] - 10s 82us/step - loss: 0.0445 - acc: 0.9832 - val_loss: 0.2834 - val_acc: 0.9361

Epoch 00083: val_acc did not improve
Epoch 84/100
83 1e-05
124800/124800 [==============================] - 10s 83us/step - loss: 0.0446 - acc: 0.9825 - val_loss: 0.2834 - val_acc: 0.9361

Epoch 00084: val_acc did not improve
Epoch 85/100
84 1e-05
124800/124800 [==============================] - 10s 83us/step - loss: 0.0450 - acc: 0.9826 - val_loss: 0.2834 - val_acc: 0.9360

Epoch 00085: val_acc did not improve
Epoch 86/100
85 1e-05
124800/124800 [==============================] - 10s 83us/step - loss: 0.0450 - acc: 0.9823 - val_loss: 0.2835 - val_acc: 0.9359

Epoch 00086: val_acc did not improve
Epoch 87/100
86 1e-05
124800/124800 [==============================] - 10s 82us/step - loss: 0.0443 - acc: 0.9831 - val_loss: 0.2835 - val_acc: 0.9360

Epoch 00087: val_acc did not improve
Epoch 88/100
87 1e-05
124800/124800 [==============================] - 10s 82